# Group 3: College ROI (Simple Baseline)

Goal: test which school characteristics are linked to median earnings 10 years after entry.


In [1]:
import pandas as pd
from davis_stats import reg, scatter

pd.set_option('display.float_format', '{:,.2f}'.format)
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm


In [2]:
# load data
file_path = "../data/raw/Most-Recent-Cohorts-Institution.csv"

df = pd.read_csv(file_path, low_memory=False)
print(df.shape)


(6429, 3306)


In [3]:
# variable labels
var_labels = {
    'MD_EARN_WNE_P10': 'Median Earnings (10 Years After Entry)',
    'MEDIAN_HH_INC': 'Median Household Income',
    'stem_share': 'STEM Enrollment Share',
    'completion_rate': 'Completion Rate',
    'ADM_RATE': 'Admission Rate',
    'prestige_proxy': 'Prestige Proxy (1 - Admission Rate)'
}


In [4]:
# keep needed columns
cols = [
    'INSTNM', 'CONTROL',
    'MD_EARN_WNE_P10', 'MEDIAN_HH_INC',
    'C150_4', 'C150_L4', 'ADM_RATE',
    'PCIP11', 'PCIP14', 'PCIP15', 'PCIP26', 'PCIP27', 'PCIP40', 'PCIP41'
]

df = df[cols].copy()

# make numeric columns numeric
num_cols = [
    'MD_EARN_WNE_P10', 'MEDIAN_HH_INC',
    'C150_4', 'C150_L4', 'ADM_RATE',
    'PCIP11', 'PCIP14', 'PCIP15', 'PCIP26', 'PCIP27', 'PCIP40', 'PCIP41'
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')


In [5]:
# build features
stem_cols = ['PCIP11', 'PCIP14', 'PCIP15', 'PCIP26', 'PCIP27', 'PCIP40', 'PCIP41']
df['stem_share'] = df[stem_cols].fillna(0).sum(axis=1)
df['completion_rate'] = df['C150_4'].fillna(df['C150_L4'])

# simple prestige proxy: lower admit rate = higher prestige
df['prestige_proxy'] = 1 - df['ADM_RATE']

# rows for modeling
model_df = df.dropna(subset=[
    'MD_EARN_WNE_P10', 'MEDIAN_HH_INC',
    'stem_share', 'completion_rate', 'prestige_proxy'
]).copy()

print(f'Modeling sample size: {model_df.shape[0]} institutions')

summary = model_df[['MD_EARN_WNE_P10', 'stem_share', 'completion_rate', 'MEDIAN_HH_INC', 'ADM_RATE', 'prestige_proxy']]
summary = summary.rename(columns=var_labels)
summary.describe()


Modeling sample size: 1675 institutions


,Median Earnings (10 Years After Entry),STEM Enrollment Share,Completion Rate,Median Household Income,Admission Rate,Prestige Proxy (1 - Admission Rate)
count,"1,675.00","1,675.00","1,675.00","1,675.00","1,675.00","1,675.00"
mean,"55,913.65",0.16,0.58,"63,269.68",0.73,0.27
std,"15,915.75",0.15,0.19,"11,858.57",0.22,0.22
min,"15,266.00",0.00,0.00,"16,928.18",0.00,0.00
25%,"45,987.00",0.05,0.46,"56,045.26",0.62,0.10
50%,"53,957.00",0.13,0.58,"63,240.60",0.78,0.22
75%,"63,398.50",0.22,0.70,"71,782.20",0.90,0.38
max,"143,372.00",1.00,1.00,"96,737.95",1.00,1.00


## Baseline Model
Earnings on median household income only.

In [6]:
baseline = reg(
    model_df,
    'MD_EARN_WNE_P10',
    ['MEDIAN_HH_INC'],
    silent=True
)

print(baseline.summary())


                            OLS Regression Results                            
Dep. Variable:        MD_EARN_WNE_P10   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     861.8
Date:                Wed, 18 Feb 2026   Prob (F-statistic):          3.73e-153
Time:                        16:08:34   Log-Likelihood:                -18234.
No. Observations:                1675   AIC:                         3.647e+04
Df Residuals:                    1673   BIC:                         3.648e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          6399.4062   1715.961      3.729

## Full Model
Adds STEM share, completion, and prestige proxy.

In [7]:
full = reg(
    model_df,
    'MD_EARN_WNE_P10',
    ['MEDIAN_HH_INC', 'stem_share', 'completion_rate', 'prestige_proxy'],
    silent=True
)

print(full.summary())


                            OLS Regression Results                            
Dep. Variable:        MD_EARN_WNE_P10   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     421.7
Date:                Wed, 18 Feb 2026   Prob (F-statistic):          2.74e-251
Time:                        16:08:34   Log-Likelihood:                -17997.
No. Observations:                1675   AIC:                         3.600e+04
Df Residuals:                    1670   BIC:                         3.603e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5357.2930   1516.500     

## Robust SE Check
HC3 robust standard errors for the full model.

In [8]:
# robust SE check (HC3)
robust_full = full.get_robustcov_results(cov_type='HC3')

coef_names = full.params.index
robust_table = pd.DataFrame({
    'coef': full.params.values,
    'se_ols': full.bse.values,
    'se_hc3': robust_full.bse,
    'p_hc3': robust_full.pvalues
}, index=coef_names)

key_rows = ['MEDIAN_HH_INC', 'stem_share', 'completion_rate', 'prestige_proxy']
print('Key coefficients with robust SE:')
print(robust_table.loc[key_rows].round(4))


Key coefficients with robust SE:
                     coef   se_ols   se_hc3  p_hc3
MEDIAN_HH_INC        0.55     0.03     0.03   0.00
stem_share      31,047.01 1,882.83 2,531.40   0.00
completion_rate 14,928.09 1,676.14 2,537.18   0.00
prestige_proxy   8,399.37 1,331.63 1,607.06   0.00


In [9]:
# compare models
print('Baseline R^2:', round(baseline.rsquared, 4))
print('Full R^2:', round(full.rsquared, 4))

coef = full.params[['MEDIAN_HH_INC', 'stem_share', 'completion_rate', 'prestige_proxy']]
coef = coef.rename(index=var_labels)

print() 
print('Key full-model coefficients:')
print(coef.round(2))


Baseline R^2: 0.34
Full R^2: 0.5025

Key full-model coefficients:
Median Household Income                    0.55
STEM Enrollment Share                 31,047.01
Completion Rate                       14,928.09
Prestige Proxy (1 - Admission Rate)    8,399.37
dtype: float64


In [10]:
# practical interpretation
stem_10pp = full.params['stem_share'] * 0.10
completion_10pp = full.params['completion_rate'] * 0.10
adm_rate_10pp = (-full.params['prestige_proxy']) * 0.10
hh_10k = full.params['MEDIAN_HH_INC'] * 10000

def money(x):
    if x < 0:
        return f"-${abs(x):,.0f}"
    return f"${x:,.0f}"

print('Practical interpretation (holding other variables fixed):')
print(f"+10 percentage points STEM share -> {money(stem_10pp)} in earnings")
print(f"+10 percentage points completion rate -> {money(completion_10pp)} in earnings")
print(f"+10 percentage points admission rate -> {money(adm_rate_10pp)} in earnings")
print(f"+$10,000 median household income -> {money(hh_10k)} in earnings")


Practical interpretation (holding other variables fixed):
+10 percentage points STEM share -> $3,105 in earnings
+10 percentage points completion rate -> $1,493 in earnings
+10 percentage points admission rate -> -$840 in earnings
+$10,000 median household income -> $5,481 in earnings


In [11]:
# coefficient chart
coef = full.params[['MEDIAN_HH_INC', 'stem_share', 'completion_rate', 'prestige_proxy']]
coef = coef.rename(index=var_labels)
coef.plot(kind='barh', figsize=(8, 5))
plt.axvline(0, color='gray', linestyle='--')
plt.title('Full Model: Key Coefficients')
plt.xlabel('Coefficient')
plt.ylabel('')
plt.tight_layout()
plt.show()


/var/folders/5k/8j9r_tw11ns72lc08wnnjy_r0000gn/T/ipykernel_14093/891307941.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
# R^2 chart
labels = ['Baseline', 'Full']
values = [baseline.rsquared, full.rsquared]
plt.figure(figsize=(6, 5))
plt.bar(labels, values)
plt.ylim(0, 1)
plt.title('Model Fit Comparison (R^2)')
plt.ylabel('R^2')
plt.tight_layout()
plt.show()


/var/folders/5k/8j9r_tw11ns72lc08wnnjy_r0000gn/T/ipykernel_14093/2980385230.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
# stem share chart
scatter(model_df, 'MD_EARN_WNE_P10', 'stem_share', fit_line=True)
plt.title(f'STEM Share vs Earnings (coef: {full.params["stem_share"]:.2f})')
plt.tight_layout()
plt.show()


/Users/jacksonmaroon/3-dimension-analytics-college-roi/.venv/lib/python3.11/site-packages/davis_stats/visualization/scatter.py:128: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/5k/8j9r_tw11ns72lc08wnnjy_r0000gn/T/ipykernel_14093/2254332006.py:5: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
# admission rate chart
adm_rate_coef = -full.params['prestige_proxy']
scatter(model_df, 'MD_EARN_WNE_P10', 'ADM_RATE', fit_line=True)
plt.title(f'Admission Rate vs Earnings (coef: {adm_rate_coef:.2f})')
plt.tight_layout()
plt.show()


/var/folders/5k/8j9r_tw11ns72lc08wnnjy_r0000gn/T/ipykernel_14093/2395561416.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Slide Regression Table
Single table output for presentation slide.

In [15]:
# slide regression table (table only)
slide_rows = ['MEDIAN_HH_INC', 'stem_share', 'completion_rate', 'prestige_proxy']

baseline_se_hc3 = baseline.get_robustcov_results(cov_type='HC3')
full_se_hc3 = full.get_robustcov_results(cov_type='HC3')

b = pd.DataFrame({
    'var': baseline.params.index,
    'coef_baseline': baseline.params.values,
    'se_baseline_hc3': baseline_se_hc3.bse,
    'p_baseline_hc3': baseline_se_hc3.pvalues
}).set_index('var')

f = pd.DataFrame({
    'var': full.params.index,
    'coef_full': full.params.values,
    'se_full_hc3': full_se_hc3.bse,
    'p_full_hc3': full_se_hc3.pvalues
}).set_index('var')

table = b.join(f, how='outer')
table = table.loc[slide_rows]
table = table.rename(index=var_labels)

table


,coef_baseline,se_baseline_hc3,p_baseline_hc3,coef_full,se_full_hc3,p_full_hc3
var,,,,,,
Median Household Income,0.78,0.03,0.00,0.55,0.03,0.00
STEM Enrollment Share,NaN,NaN,NaN,"31,047.01","2,531.40",0.00
Completion Rate,NaN,NaN,NaN,"14,928.09","2,537.18",0.00
Prestige Proxy (1 - Admission Rate),NaN,NaN,NaN,"8,399.37","1,607.06",0.00


In [16]:
# export slide table
import os
os.makedirs('../outputs/tables', exist_ok=True)
table.round(4).to_csv('../outputs/tables/slide_regression_table.csv')


## Diagnostics
Required checks for methods report.

In [17]:
# diagnostics: BP, Shapiro, VIF, Durbin-Watson
print('Breusch-Pagan test (full model residuals)')
bp = het_breuschpagan(full.resid, full.model.exog)
print({'LM stat': bp[0], 'LM p-value': bp[1], 'F stat': bp[2], 'F p-value': bp[3]})

print()
print('Shapiro-Wilk test (full model residuals)')
sh = shapiro(full.resid)
print({'W': sh.statistic, 'p-value': sh.pvalue})

print()
print('Durbin-Watson (full model residuals)')
print(sm.stats.stattools.durbin_watson(full.resid))

print()
print('VIF values')
X = model_df[['MEDIAN_HH_INC', 'stem_share', 'completion_rate', 'prestige_proxy']].copy()
X = sm.add_constant(X)
vif = []
for i, col in enumerate(X.columns):
    if col == 'const':
        continue
    vif.append((col, variance_inflation_factor(X.values, i)))
print(vif)


Breusch-Pagan test (full model residuals)
{'LM stat': np.float64(35.0889286190151), 'LM p-value': np.float64(4.454000432751991e-07), 'F stat': np.float64(8.933184215960107), 'F p-value': np.float64(3.886577274314572e-07)}

Shapiro-Wilk test (full model residuals)
{'W': np.float64(0.9153602222738604), 'p-value': np.float64(1.3075269640005668e-29)}

Durbin-Watson (full model residuals)
1.7954657957538633

VIF values
[('MEDIAN_HH_INC', np.float64(1.2688353099867846)), ('stem_share', np.float64(1.110901138115563)), ('completion_rate', np.float64(1.3352358785649572)), ('prestige_proxy', np.float64(1.1743233248361304))]
